In [1]:
%run stdPackages.ipynb
slides = False # print to slides format if True
out_folder = os.path.join(d['curr'], 'Misc', 'Figs')
d['data'] = os.path.join(d['curr'],'Misc','Data')
read = {'variables': ['Fundamentals', 'Load', 'Generators_Other','TL'], 
        'variable2D': ['Generators_FuelMix','HourlyVariation'],
        'scalars': ['Scalars'],
        'maps': ['Generators_Categories', 'Load_Categories']}
db = dbFromWB(os.path.join(d['data'],'mBasicTrade1.xlsx'), read)
readSets(db)

Set up model:

In [2]:
m = mBasicTrade.mBasicTrade(db)
m.solve()

Solution status 0: Optimization terminated successfully.


### LDC and RDC

Print the same plots as for ```mBasicInt``` for the geographic area $g2$. The area $g1$ is identical to plots in ```mBasicInt```.

Load duration curve (LDC) and residual demand curve (RDC):

In [3]:
LDC = m.hourlyLoad.sort_values(ascending=False).xs('g2',level='g')
LDC.index = [i/(len(LDC)) for i in range(1, len(LDC)+1)]
LDC.at[0] = LDC.iloc[0]
RDC = (m.hourlyLoad-m.hourlyGeneratingCapacity.xs('id6')).sort_values(ascending = False)
RDC.index = [i/(len(RDC)) for i in range(1, len(RDC)+1)]
RDC.at[0] = RDC.iloc[0]

In [4]:
%%capture
mult_graphs()
nplots = 4
nrows = math.ceil(nplots/2)
fig, axes = plt.subplots(nrows, min(nplots, 2), figsize = (14, (4*nrows)));
plt.subplots_adjust(hspace=0.35)
# Plot 1: Hourly load
ax = plt.subplot(nrows, min(nplots,2), 1)
m.hourlyLoad.xs('g2',level='g').plot.bar(ax = ax, legend = False);
ax.set_xlabel(r'$h$', labelpad=10);
ax.set_ylabel(r"Load, GJ", labelpad=10);
ax.set_title('Hourly load');

# Plot 2: Load duration curve
ax = plt.subplot(nrows, min(nplots,2), 2)
seaborn.lineplot(data=LDC, ax = ax, linewidth = 3, legend = False);
ax.set_xlim([0,1]);
ax.set_xlabel(r'Capacity factor', labelpad=10);
ax.set_ylabel(r"Load, GJ",labelpad=10);
ax.set_title('Load duration curve');

# Plot 3: Hourly Variation in Generation:
ax = plt.subplot(nrows, min(nplots,2), 3)
m.hourlyGeneratingCapacity.xs('id6').plot.bar(ax = ax, legend = False,linewidth=None);
ax.set_xlabel(r'$h$', labelpad=10);
ax.set_ylabel(r"Generating capacity, GJ", labelpad=10);
ax.set_title('Generating capacity, PV');

# Plot 4: Residual demand curve
ax = plt.subplot(nrows, min(nplots,2), 4)
seaborn.lineplot(data=RDC, ax = ax, linewidth = 3, legend = False);
ax.set_xlim([0,1]);
ax.set_xlabel(r'Capacity factor', labelpad=10);
ax.set_ylabel(r"Demand net of PV, GJ",labelpad=10);
ax.set_title('Residual demand curve');
fig.tight_layout()
if slides:
    fig.savefig(f"{out_folder}\\mBasicTrade_hvt_g2_slides.pdf",facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(f"{out_folder}\\mBasicTrade_hvt_g2.pdf",edgecolor='k')

### Plot merit order curve for each hour:

In [5]:
def demand_h(m,h):
    return standardPlots.demandLinear_df(m.db['MWP_LoadShedding'], m.hourlyLoad.xs('g2',level='g').xs(h))
def supply_h(m,h,maxY=30):
    return standardPlots.meritOrderCurve_df(lpCompiler.broadcast(m.db['mc'], m.db['id2g']).xs('g2',level='g'), lpCompiler.broadcast(m.hourlyGeneratingCapacity, m.db['id2g']).xs('g2',level='g').xs(h,level='h'),maxY=maxY)

In [6]:
%%capture
mult_graphs()
nplots = len(m.db['h'])
nrows = math.ceil(nplots/2)
fig,axes = plt.subplots(nrows,min(nplots,2),figsize=(14,(4*nrows)));
plt.subplots_adjust(hspace=0.35)
offset_N, offset_Arrow, offset_q = 1, 1, 3
for j in range(nplots):
    ax = plt.subplot(nrows,min(nplots,2),j+1)
    h = j+1
    df = supply_h(m,h)
    dh = demand_h(m,h)
    
    df.plot(linewidth=3,ax=ax,legend=False);
    dh.plot(linewidth=3,ax=ax,legend=False);
    ax.set_xlabel(r'$GJ$', labelpad = 5);
    ax.set_ylabel(r'$€/$GJ', labelpad = 5);
    ax.set_xlim([0, max([dh.index.max(), df.index.max()])+5]);
    ax.set_ylim([0, math.ceil(max([df.max()[0], dh.max()[0]]))]);
    ax.set_title(f"""$h={h}$""")
    
    if j>0:
        plt.text(df.index[2]/2-8, df.iloc[2]+offset_N, f'Solar');
        plt.text(df.index[2]/2-5, df.iloc[2]-offset_q, f'$q_S$');
        plt.annotate("",xy=(1, df.iloc[2]-offset_Arrow), xytext=(df.index[2]-1, df.iloc[2]-offset_Arrow), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));
        plt.annotate("",xy=(df.index[2]-1, df.iloc[2]-offset_Arrow), xytext=(1, df.iloc[2]-offset_Arrow), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));
    
    plt.text((df.index[4]+df.index[2])/2-8, df.iloc[4]+offset_N, f'Gas')
    plt.text((df.index[4]+df.index[2])/2-5, df.iloc[4]-offset_q, f'$q_G$');
    plt.annotate("",xy=(df.index[2]+1, df.iloc[4]-offset_Arrow), xytext=(df.index[4]-1,df.iloc[4]-offset_Arrow), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));
    plt.annotate("",xy=(df.index[4]-1, df.iloc[4]-offset_Arrow), xytext=(df.index[2]+1, df.iloc[4]-offset_Arrow), arrowprops=dict(width=1, color='k', headwidth=5, headlength=15));
    
fig.legend(['Generation', 'Load'],loc=9,ncol=2,frameon=True)
fig.tight_layout();
fig.subplots_adjust(top=0.9);
if slides:
    fig.savefig(f"{out_folder}\\mBasicTrade_MeritOrder_g2_slides.pdf",facecolor='#FAFAFA',edgecolor='k')
else:
    fig.savefig(f"{out_folder}\\mBasicTrade_MeritOrder_g2.pdf",edgecolor='k')